In [23]:
import pandas as pd
import os
from scipy.stats import chi2_contingency

In [ ]:
#random notes and settings
#set exporter.fhir.export = false
#set exporter.csv.export = true

#encounters want reason code == 55680006 (drug overdose)
#encounters want description == 'Death Certification'

#run_synthea -p 10000 -s 10000 -cs 12345 -m opioid_addiction Maine Bangor

#^^this command generates 10000 people (-p) with the seed 10000 (-s) and the provider seed of 12345 (-cs) using the opiod_addition module (-m) in Bangor, ME

In [ ]:
def makeEncountersDF(path, seeds):
    '''
    Go to the path and grab all encounters.csv and put them in one file with an extra column for seed
    '''
    df = pd.DataFrame()
    for seed in seeds:
        try:
            encounters = pd.read_csv(os.path.join(path, 'bangor_s' + str(seed), 'encounters.csv'), dtype=str)
            encounters['seed'] = seed
            df = df.append(encounters)
        except:
            print('File for seed', str(seed), 'does not exist...skipping')
    return df

In [ ]:
def getODEncounters(df):
    '''
    Return all drug overdose encounters (reason code 5568006) from a given encounters dataframe
    This will include overdose deaths as well -- description -- 'Death Certification'
    '''
    data = df[df['REASONCODE'] == '55680006']
    return data

In [ ]:
def getODDeaths(df):
    '''
    Return all drug overdose deaths (reason code 5568006) from a given encounters dataframe
    DESCRIPTION == 'Death Certification' and REASONCODE == '5568006'
    '''
    data = df[(df['DESCRIPTION'] == 'Death Certification') & (df['REASONCODE'] == '55680006')]
    return data

In [ ]:
def getODstats(df):
    '''
    get patient level sample statistics on probability of death per drug overdose ED visit
    '''
    #getting all overdose encouunters
    od_enc = getODEncounters(df)
    od_enc = od_enc.groupby(['PATIENT','seed'], as_index=False)['REASONCODE'].count().rename(columns={'REASONCODE':'OD_count'})
    #getting all overdose deaths
    od_death = getODDeaths(df)
    od_death = od_death.groupby(['PATIENT','seed'], as_index=False)['REASONCODE'].count().rename(columns={'REASONCODE':'OD_death'})
    #joining the above two dataframes
    od = pd.merge(od_enc, od_death, how='left', on=['PATIENT', 'seed']).fillna(0)
    #calculating patient level probability of death from overdose encounter
    od['prDeath'] = od['OD_death']/od['OD_count']
    #making column for weight of patient to calculate weighted average probability
    od['weight'] = od['OD_count']/sum(od['OD_count'])
    #weighted pr(death) -- can sum this column to get weighted sample pr(death)
    od['weightedPrDeath'] = od['weight']*od['prDeath']
    
    return od

In [ ]:
# path = r'C:\repos\Synthea\output'
path = r'\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\SyntheaChallenge\data'
seeds = [10000, 13370, 22222, 23123, 33555, 39093, 45000, 51327, 65888, 74982]
#seeds = [12345]
# seeds = [22222]

#pull in data
df = makeEncountersDF(path, seeds)

In [ ]:
#calculate overdose stats
od_df = getODstats(df)

print(od_df['prDeath'].mean())

print(od_df['weightedPrDeath'].sum())

In [ ]:
len(df['PATIENT'].unique())

In [ ]:
od_df

## The number of prescriptions per person, per year, by drug code & description

In [5]:
def makeMedicationsDF(path, seeds, data_source=''):
    '''
    Go to the path and grab all encounters.csv and put them in one file with an extra column for seed
    '''
    if data_source == 'LMISynthea':
        data_source = '_' + data_source
        
    df = pd.DataFrame()
    for seed in seeds:
        try:
            medications = pd.read_csv(os.path.join(path, 'bangor_s' + str(seed) + data_source, 'medications.csv'), dtype=str)
            medications['seed'] = seed
            df = df.append(medications)
        except:
            print('File for seed', str(seed), 'does not exist...skipping')
    return df

In [6]:
path = r'\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\SyntheaChallenge\data'
# path = r'C:\repos\Synthea\output'
seeds = [10000, 13370, 22222, 23123, 33555, 39093, 45000, 51327, 65888, 74982]

# pull in data
df_legacy_Synthea = makeMedicationsDF(path, seeds)
df_LMI_Synthea = makeMedicationsDF(path, seeds, data_source='LMISynthea')

In [7]:
df_legacy_Synthea

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION,seed
0,2010-07-21T21:17:12Z,2011-07-21T21:17:12Z,cb8ec237-0759-b2e4-e89a-79df87ae36a0,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,e554e4c1-e02a-6842-6d84-f33c68e67330,807283,Mirena 52 MG Intrauterine System,681.13,621.13,12,8173.56,NaN,NaN,10000
1,2011-09-26T06:40:20Z,2012-09-20T06:40:20Z,956e9fa8-ed18-234d-f77b-bdc9c93fe5da,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,eb3d2956-efa3-e324-73af-d389237b1388,749762,Seasonique 91 Day Pack,25.97,0.00,12,311.64,NaN,NaN,10000
2,2001-06-26T12:17:19Z,2002-07-02T12:17:19Z,a36a5420-69d5-6356-2a07-d0fa177c59ef,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,9af04fe6-e9ce-4085-20ce-c50b99a1c53c,308136,amLODIPine 2.5 MG Oral Tablet,0.01,0.00,371,3.71,59621000,Hypertension,10000
3,2001-07-26T12:17:19Z,2002-07-02T12:17:19Z,a36a5420-69d5-6356-2a07-d0fa177c59ef,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,87d37361-f626-29b4-820d-46972c1721ef,314076,lisinopril 10 MG Oral Tablet,0.01,0.00,341,3.41,59621000,Hypertension,10000
4,2002-07-02T12:17:19Z,2003-07-08T12:17:19Z,a36a5420-69d5-6356-2a07-d0fa177c59ef,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,d2ab512a-d049-23b6-078e-e8b57728abd1,314076,lisinopril 10 MG Oral Tablet,0.01,0.00,371,3.71,59621000,Hypertension,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955697,2020-05-30T15:54:09Z,2021-06-11T15:50:14Z,414e1d64-2b15-4026-82f9-0b9db7193669,b3221cfc-24fb-339e-823d-bc4136cbc4ed,53cff874-3a38-fd4d-bab9-5d20ed5a8282,197361,Amlodipine 5 MG Oral Tablet,36.18,0.00,12,434.16,NaN,NaN,74982
1955698,2021-06-11T15:50:14Z,NaN,414e1d64-2b15-4026-82f9-0b9db7193669,b3221cfc-24fb-339e-823d-bc4136cbc4ed,a0e28916-e357-afc9-a9c1-79d3a5ae1989,309362,Clopidogrel 75 MG Oral Tablet,38.95,0.00,1,38.95,NaN,NaN,74982
1955699,2021-06-11T15:50:14Z,NaN,414e1d64-2b15-4026-82f9-0b9db7193669,b3221cfc-24fb-339e-823d-bc4136cbc4ed,a0e28916-e357-afc9-a9c1-79d3a5ae1989,705129,Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray,185.59,0.00,1,185.59,NaN,NaN,74982
1955700,2021-06-11T15:50:14Z,NaN,414e1d64-2b15-4026-82f9-0b9db7193669,b3221cfc-24fb-339e-823d-bc4136cbc4ed,a0e28916-e357-afc9-a9c1-79d3a5ae1989,312961,Simvastatin 20 MG Oral Tablet,33.43,0.00,1,33.43,NaN,NaN,74982


In [8]:
df_LMI_Synthea

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION,seed
0,1977-11-24T06:10:36Z,1978-05-19T06:10:36Z,17f1d629-664c-4027-584a-1d9dd72c7316,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,cbe27604-ad98-5055-eb43-8767f3045f0f,197591,Diazepam 5 MG Oral Tablet,1197.45,0.00,5,5987.25,NaN,NaN,10000
1,1978-07-20T03:55:19Z,NaN,17f1d629-664c-4027-584a-1d9dd72c7316,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,5374d1d4-4b54-d284-a418-427b8e7cf246,204892,clonazePAM 0.25 MG Oral Tablet,87.23,0.00,13,1133.99,NaN,NaN,10000
2,2012-10-07T14:48:07Z,2012-11-17T17:17:07Z,c78e5e78-1084-5cec-4a02-ece808a59013,047f6ec3-6215-35eb-9608-f9dda363a44c,653b8d4c-40ed-535d-ba06-d5b5559429ae,310965,Ibuprofen 200 MG Oral Tablet,13.53,0.00,1,13.53,NaN,NaN,10000
3,2012-10-14T17:17:07Z,2012-11-10T13:37:07Z,c78e5e78-1084-5cec-4a02-ece808a59013,047f6ec3-6215-35eb-9608-f9dda363a44c,2492a12b-82d6-c9db-f0a5-f65df7450b73,861467,Meperidine Hydrochloride 50 MG Oral Tablet,17.73,0.00,1,17.73,NaN,NaN,10000
4,2010-10-14T08:09:39Z,2011-10-09T08:09:39Z,956e9fa8-ed18-234d-f77b-bdc9c93fe5da,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ed50c3d0-d03d-0490-2510-03bcb2be43c2,757594,Jolivette 28 Day Pack,26.17,0.00,12,314.04,NaN,NaN,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944801,2020-06-15T00:33:18Z,2021-06-21T00:33:18Z,995af99c-c416-af93-e7a7-28810e89f127,b3221cfc-24fb-339e-823d-bc4136cbc4ed,38c0178b-3fdc-aeec-ec19-6ea71053c012,855332,Warfarin Sodium 5 MG Oral Tablet,121.21,21.21,12,1454.52,NaN,NaN,74982
1944802,2021-06-21T00:33:18Z,NaN,995af99c-c416-af93-e7a7-28810e89f127,b3221cfc-24fb-339e-823d-bc4136cbc4ed,1a0631b8-fa35-07b6-9d4b-ea7132f6e7a8,206905,Ibuprofen 400 MG Oral Tablet [Ibu],170.46,0.00,3,511.38,NaN,NaN,74982
1944803,2021-06-21T00:33:18Z,NaN,995af99c-c416-af93-e7a7-28810e89f127,b3221cfc-24fb-339e-823d-bc4136cbc4ed,1a0631b8-fa35-07b6-9d4b-ea7132f6e7a8,897718,Verapamil Hydrochloride 40 MG,148.93,0.00,1,148.93,NaN,NaN,74982
1944804,2021-06-21T00:33:18Z,NaN,995af99c-c416-af93-e7a7-28810e89f127,b3221cfc-24fb-339e-823d-bc4136cbc4ed,1a0631b8-fa35-07b6-9d4b-ea7132f6e7a8,197604,Digoxin 0.125 MG Oral Tablet,32.15,0.00,1,32.15,NaN,NaN,74982


In [9]:
def add_year(df):
    df['YEAR'] = df['START'].str.slice(stop=4)
    return df

In [10]:
def num_prescription_analysis(df, data_source='legacy'):
    df_grouped = df.groupby(['seed', 'PATIENT', 'YEAR', 'CODE', 'DESCRIPTION'])['ENCOUNTER'].count() \
        .reset_index(name='Number of Prescriptions')
    
    df_grouped.to_csv("{}_Synthea_prescription_info_including_seeds.csv".format(data_source), index=False)
    
    listOfDrugs = df_grouped.groupby(['CODE','DESCRIPTION'])['Number of Prescriptions'].sum().reset_index()
    listOfDrugs.to_csv("{}_Synthea_prescription_list.csv".format(data_source), index=False)

In [11]:
df_legacy_Synthea = add_year(df_legacy_Synthea)
df_LMI_Synthea = add_year(df_LMI_Synthea)

## How many opioids were prescribed by year divided by total number of people in the simulation in the year

### need to modify this to keep the seed and use the list of opioids that Maureen is filling out

In [12]:
opioid_prescriptions = '''1 ML Morphine Sulfate 5 MG/ML Injection
10 ML Alfentanil 0.5 MG/ML Injection
10 ML Fentanyl 0.05 MG/ML Injection
12 HR Hydrocodone Bitartrate 10 MG Extended Release Oral Capsule
5 ML SUFentanil 0.05 MG/ML Injection
72 HR Fentanyl 0.025 MG/HR Transdermal System
Abuse-Deterrent 12 HR Oxycodone Hydrochloride 10 MG Extended Release Oral Tablet [Oxycontin]
Abuse-Deterrent 12 HR Oxycodone Hydrochloride 15 MG Extended Release Oral Tablet
Acetaminophen 300 MG / Codeine Phosphate 15 MG Oral Tablet
Acetaminophen 300 MG / HYDROcodone Bitartrate 5 MG Oral Tablet
Acetaminophen 300 MG / Hydrocodone Bitartrate 5 MG Oral Tablet
Acetaminophen 325 MG / HYDROcodone Bitartrate 7.5 MG Oral Tablet
Acetaminophen 325 MG / Oxycodone Hydrochloride 10 MG Oral Tablet [Percocet]
Acetaminophen 325 MG / oxyCODONE Hydrochloride 2.5 MG Oral Tablet
Acetaminophen 325 MG / oxyCODONE Hydrochloride 5 MG Oral Tablet
Acetaminophen 325 MG / Oxycodone Hydrochloride 5 MG Oral Tablet
buprenorphine 2 MG / naloxone 0.5 MG Sublingual Tablet
Meperidine Hydrochloride 50 MG Oral Tablet
methadone hydrochloride 10 MG Oral Tablet
remifentanil 2 MG Injection
tramadol hydrochloride 50 MG Oral Tablet'''.lower().split("\n")

In [13]:
def prescription_per_capita_analysis(df, data_source='legacy'):
    df_total_people_by_year = df.groupby(['YEAR', 'seed'])['PATIENT'].nunique().reset_index(name='Total Unique Patients')
    df_opioids = df[df['DESCRIPTION'].str.lower().isin(opioid_prescriptions)]
    df_num_opioids_by_year = df_opioids.groupby(['YEAR', 'seed'])['DESCRIPTION'].count().reset_index(name='Number of Opioids')
    
    df_opioids_per_capita = df_num_opioids_by_year.merge(df_total_people_by_year, how='left', on=['YEAR', 'seed'])
    df_opioids_per_capita['Per Capita'] = df_opioids_per_capita['Number of Opioids'] / df_opioids_per_capita['Total Unique Patients']
    
#     df_opioids_per_capita.to_csv("{}_Synthea_opioids_prescription_per_capita.csv".format(data_source), index=False)
    return df_opioids_per_capita

In [14]:
df_opioids_per_capita_legacy = prescription_per_capita_analysis(df_legacy_Synthea)
df_opioids_per_capita_LMI = prescription_per_capita_analysis(df_LMI_Synthea)

In [15]:
df_opioids_per_capita_legacy

,YEAR,seed,Number of Opioids,Total Unique Patients,Per Capita
0,1922,13370,1,25,0.040000
1,1922,51327,1,32,0.031250
2,1923,13370,1,33,0.030303
3,1923,22222,2,36,0.055556
4,1923,23123,2,36,0.055556
...,...,...,...,...,...
980,2021,39093,4926,11975,0.411357
981,2021,45000,4916,11985,0.410179
982,2021,51327,4904,12258,0.400065
983,2021,65888,4751,11969,0.396942


In [16]:
df_opioids_per_capita_legacy['YEAR'].value_counts().sort_index()

1922     2
1923     7
1924     8
1925     9
1926    10
        ..
2017    10
2018    10
2019    10
2020    10
2021    10
Name: YEAR, Length: 100, dtype: int64

In [17]:
df_opioids_per_capita_LMI

,YEAR,seed,Number of Opioids,Total Unique Patients,Per Capita
0,1923,13370,3,36,0.083333
1,1923,22222,2,35,0.057143
2,1923,23123,1,34,0.029412
3,1923,33555,1,19,0.052632
4,1923,51327,2,32,0.062500
...,...,...,...,...,...
981,2021,39093,5234,12425,0.421247
982,2021,45000,5011,12582,0.398267
983,2021,51327,5065,12521,0.404520
984,2021,65888,5042,12452,0.404915


In [18]:
df_opioids_per_capita_LMI['YEAR'].value_counts().sort_index()

1923     7
1924     9
1925    10
1926    10
1927    10
        ..
2017    10
2018    10
2019    10
2020    10
2021    10
Name: YEAR, Length: 99, dtype: int64

## Comparison

In [30]:
def extract_comparison_lists(df_legacy, df_LMI, year):
    result_list = []
    
    for df in (df_legacy, df_LMI):
        lst = list(df[df['YEAR'] == year]['Per Capita'])
        result_list.append(lst)
    
    return result_list

In [26]:
def chi_square_validation(comparison_list, confidence=0.99):
    stat, p, dof, expected = chi2_contingency(comparison_list)

    # interpret p-value
    alpha = 1 - confidence
    print("p value is " + str(p))
    if p <= alpha:
        print('the two groups have significant difference (reject H0)')
    else:
        print('the two groups have no significant difference (H0 holds true)')

### LMI Synthea vs. Legacy Synthea

In [36]:
comparison_list = extract_comparison_lists(df_opioids_per_capita_legacy, df_opioids_per_capita_LMI, '2019')

comparison_list

[[0.44415521812484937,
  0.45470266115861274,
  0.43020039788441944,
  0.4543671673203865,
  0.42502059208294973,
  0.4381118542211883,
  0.43227331950007236,
  0.43761345180089806,
  0.426984050960295,
  0.45488304798649626],
 [0.43287115319580655,
  0.4356808418207269,
  0.43660817585807943,
  0.453416149068323,
  0.4538840760063179,
  0.4581847962231429,
  0.4402974264883395,
  0.44048197454086097,
  0.4471718249733191,
  0.44392232078638216]]

In [33]:
chi_square_validation(comparison_list)

p value is 0.9999999999999984
the two groups have no significant difference (H0 holds true)


### LMI Synthea vs. Ground Truth

In [37]:
comparison_list = [comparison_list[1], [0.47] * 10]

comparison_list

[[0.43287115319580655,
  0.4356808418207269,
  0.43660817585807943,
  0.453416149068323,
  0.4538840760063179,
  0.4581847962231429,
  0.4402974264883395,
  0.44048197454086097,
  0.4471718249733191,
  0.44392232078638216],
 [0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47, 0.47]]

In [38]:
chi_square_validation(comparison_list)

p value is 1.0
the two groups have no significant difference (H0 holds true)
